***Finding a DNA sub-sequence in a DNA sequence data***

***Query a pattern: p, if it's included in the text: t***

**Firstly,** I create a list contained all the substrings of t with length in k.

t= ATCGCGTCG, k=3 --> t_list= [ATC, TCG, CGC, GCG, CGT, GTC, TCG]

**Secondly,** I will match the first k-mer of p.

p= TCGC, p_first_kmer = TCG --> match = t_list[1] and t_list[6]

**Finally,** I will match the rest of p.

p_rest = C --> if t[1+3] == C, then hit; if t[6+3]== C, then hit.



In [2]:
import bisect
class Index(object):
    """ Holds a substring index for a text T """

    def __init__(self, t, k):
        """ Create index from all substrings of t of length k """
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer

    def query(self, p):
        """ Return index hits for first k-mer of p """
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [3]:
def queryIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:  # verify that rest of P matches
            offsets.append(i)
    return offsets

In [4]:
t = 'ACTTGGAGATCTTTGAGGCTAGGTATTCGGGATCGAAGCTCATTTCGGGGATCGATTACGATATGGTGGGTATTCGGGA'
p = 'GGTATTCGGGA'

index = Index(t, 4)
print(queryIndex(p, t, index)) # 21, 68

[21, 68]


In [1]:
#upload a reference genome data in fasta format
from google.colab import files
uploaded = files.upload()

Saving chr1.GRCh38.excerpt.fasta to chr1.GRCh38.excerpt.fasta


In [6]:
#read genome in fasta format
def readGenome(filename):
    genome=''
    with open (filename,'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
    
genome = readGenome('chr1.GRCh38.excerpt.fasta')
p='GGCGCGGTGGCTCACGCCTGTAAT'
t= genome

In [7]:
# using pigeonhole principle to query sequences with mismatches 
allow_mm = 2
segment_length = int(round(len(p) / (allow_mm+1)))
all_matches = set()
totalIndexHits = 0

for i in range(allow_mm + 1):
    start = i*segment_length
    end = min((i+1)*segment_length, len(p))
    index = Index(t,segment_length)
    totalIndexHits += len(index.query(p[start:end]))
    matches = queryIndex(p[start:end],t,index)

        # Extend matching segments to see if whole p matches
    for m in matches:
        if m < start or m-start+len(p) > len(t): # avoid matches at the beginning or at the end that the rest of length can't include p
            continue
        mismatches = 0
        for j in range(0, start):
            if not p[j] == t[m-start+j]:
                mismatches += 1
                if mismatches > allow_mm:
                    break
        for j in range(end, len(p)):
            if not p[j] == t[m-start+j]:
                mismatches += 1
                if mismatches > allow_mm:
                    break
        if mismatches <= allow_mm:
            all_matches.add(m - start)
print(list(all_matches))
print(len(list(all_matches)))
print(totalIndexHits)

[273669, 681737, 717706, 262042, 635931, 84641, 160162, 724927, 657496, 160729, 56922, 191452, 551134, 747359, 421221, 147558, 364263, 465647, 429299]
19
90


***Extracting substring from text t with intervals. I can quick scan through t to match the pattern p***

Ex. t= ATATAT and interval= 2, t_0 = AA; t_1 = TT, etc.

p= ATA, p_0 = AA ---> p_0 == t_0

Once we find the hit, we verify the rest of the sequence.

In [10]:
class SubseqIndex(object): 
    """ Holds a subsequence index for a text T """
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [11]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
ind = SubseqIndex(t,8,3)
allow_mm =0
offsets = set()
totalIndexHits = 0

for i in range(3): # GGCGCGGTGGCTCACGCCTGTAAT if extract 8 mer at a time with interval:3, only three patterns
  hits = ind.query(p[i:])
  totalIndexHits += len(hits) #sum the number of hits when querying each of those three patterns
  for k in hits:
    for j in range(len(p)):   #in each hits, compare each character of t and p.
      if t[k-i+j] != p[j]:
        allow_mm +=1

        if allow_mm > 2:
          allow_mm = 0
          break

      if j== len(p)-1 :       #when I read the sequence to the end and the mismatches is lower than 2: record this hit
          offsets.add(k-i)
          allow_mm = 0
          
print(len(offsets),totalIndexHits) 

19 79
